In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
import plotly.express as px
%matplotlib inline


# 한글 폰트 설정
rc('font', family='AppleGothic')

plt.rcParams['axes.unicode_minus'] = False

## 0. 로스터 변경사항 

- In

    안치홍 (내야수, 롯데) / 이상규 (투수, LG) / 배민서 (투수, NC) / 김강민 (외야수, SSG) / 이재원 (포수, SSG) / 김도빈 (투수, 수원)

- Out

    오선진 (내야수)

- 계약 종료

    이재민, 이석제, 장웅정, 천보웅, 신현수, 이준기, 김재영, 류원석, 류희운, 박윤철, 송윤준 (이상 투수) / 고영재, 원혁재, 이정재, 노수광, 유상빈, 장운호


신인드래프트에선 1-2픽 투수, 3-4픽 내야수를 영입했다.

---

# 1. 분석 포인트

전개 방향

1. 득점 / 실점 (Q0에서 어떻게 살펴봤는지 보자)
2. 타자는 포지션별로! (타율 / 타점)
3. 수비는 포지션별로! (실책)
4. 전체적인 WAR 지표로 
5. 투수는 선발 vs 중계 vs 마무리
6. 방어율/승/홀드/세이브 등등등..
7. WAR

그러면 지금 뭐가 부족한지 알 수 있을 것이다!

새로 들어온 선수는 이걸 채울수 있느냐?
아니라면, 새로 들어온 선수를 포함한 기록 측정을 다시 하였을 때 각 기록이 상승이 되느냐? 


---

In [4]:
hitter = pd.read_csv('./dataset/hitterTotal.csv', index_col= False)
pitcher = pd.read_csv('./dataset/pitcherTotal.csv', index_col= False)

In [213]:
hitter = hitter[hitter['포지션'] != '투수']

In [214]:
hitter.loc[hitter['포지션']=='지명타자', '팀명'].unique()

array(['SSG', 'NC', '한화', 'KT', '삼성', '롯데', 'KIA', 'LG', '두산'],
      dtype=object)

In [215]:
pitcher[(pitcher['포지션'] != '선발') & (pitcher['포지션'] != '중계')]

,이름,id,연도,팀명,평균자책점,경기,완투,완봉,승리,패배,...,생년월일,포지션,WAR,폭투,ERA,FIP,WHIP,ERA+,FIP+,WPA


In [216]:
#hitter.to_csv('hitter.csv', index=False)
pitcher.to_csv('pitcher.csv', index=False)

# 2. 분석

In [3]:
hitter = pd.read_csv('hitter.csv', index_col=False)
pitcher = pd.read_csv('pitcher.csv', index_col=False)

In [4]:
pitcher.columns

Index(['이름', 'id', '연도', '팀명', '평균자책점', '경기', '완투', '완봉', '승리', '패배', '세이브',
       '홀드', '승률', '타자수', '이닝', '피안타', '피홈런', '볼넷', '사구', '삼진', '실점', '자책점',
       '생년월일', '포지션', 'WAR', '폭투', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+',
       'WPA'],
      dtype='object')

In [5]:
hitter.columns

Index(['이름', 'id', '연도', '팀명', '타율', '경기', '타석', '타수', '득점', '안타', '2루타',
       '3루타', '홈런', '루타', '타점', '도루', '도루실패', '볼넷', '사구', '삼진', '병살타', '장타율',
       '출루율', '실책', '생년월일', '포지션', '희생플라이', 'WAR*'],
      dtype='object')

- 타격 (수치)

    '이름', '팀명', '포지션', '타석', '타수', '득점', '안타', '2루타', '3루타', '홈런', '루타', '타점', '볼넷', '사구', '삼진', '병살타'


- 타격 (직접 계산 비율) : '타율', '장타율', '출루율'


- 주루

    '이름', '팀명', '포지션' '도루', '도루실패'

- 수비
    
    '이름', '팀명', '포지션', '실책'

### 팀 - 포지션 별 전처리

#### 타자

In [6]:
total_WAR = hitter[['팀명', 'WAR*']]
total_WAR = total_WAR.groupby('팀명').agg({"WAR*" : 'sum'}).reset_index()
total_WAR

,팀명,WAR*
0,KIA,25.94
1,KT,19.64
2,LG,28.90
3,NC,24.90
4,SSG,22.97
5,두산,20.63
6,롯데,18.58
7,삼성,16.36
8,키움,18.10
9,한화,16.00


In [9]:
tt = hitter.loc[(hitter['포지션'] != '1루수') & (hitter['포지션'] != '3루수') , ['팀명', '타석', '타수', '득점', '안타', '2루타', '3루타', '도루', '홈런', '루타', '타점', '볼넷', '사구', '삼진', '병살타', '희생플라이']]
tt = tt.groupby('팀명').sum().reset_index()
tt['타율'] = (tt['안타'] / tt['타수']).round(3)
tt['1루타'] = tt['안타'] - tt['2루타'] - tt['3루타'] - tt['홈런']
tt['장타율'] = ((tt['1루타'] + 2*tt['2루타'] + 3*tt['3루타'] + 4*tt['홈런']) / tt['타수']).round(3)
tt['출루율'] = ((tt['안타'] + tt['볼넷'] + tt['사구']) / (tt['타수'] + tt['볼넷'] + tt['사구'] + tt['희생플라이'])).round(3)
tt['효율'] = (tt['타점'] / tt['타수']).round(3)
tt

,팀명,타석,타수,득점,안타,2루타,3루타,도루,홈런,루타,...,볼넷,사구,삼진,병살타,희생플라이,타율,1루타,장타율,출루율,효율
0,KIA,4355,3854,551,1084,182,14,80,80,1534,...,368,39,675,88,36.0,0.281,808,0.398,0.347,0.142
1,KT,4415,3883,525,1017,180,11,79,61,1402,...,413,34,834,65,38.0,0.262,765,0.361,0.335,0.120
2,LG,4394,3744,579,1028,163,18,148,58,1401,...,458,67,614,71,42.0,0.275,789,0.374,0.360,0.141
3,NC,4237,3697,538,1020,178,23,101,78,1478,...,391,64,763,89,43.0,0.276,741,0.400,0.352,0.140
4,SSG,4363,3812,490,991,168,14,67,80,1427,...,402,52,697,66,33.0,0.260,729,0.374,0.336,0.117
5,두산,4250,3699,479,933,147,29,117,68,1342,...,411,61,740,70,23.0,0.252,689,0.363,0.335,0.109
6,롯데,4429,3879,528,1045,177,15,90,56,1420,...,398,48,775,80,42.0,0.269,797,0.366,0.341,0.127
7,삼성,4117,3628,502,998,155,19,75,69,1398,...,355,50,653,83,32.0,0.275,755,0.385,0.345,0.124
8,키움,4226,3730,487,1007,180,26,51,47,1380,...,358,61,799,63,40.0,0.270,754,0.370,0.340,0.111
9,한화,4072,3546,414,821,148,11,64,39,1108,...,374,62,851,70,31.0,0.232,623,0.312,0.313,0.096


In [10]:
tt = hitter.loc[(hitter['포지션'] != '1루수') & (hitter['포지션'] != '3루수') ,['팀명', 'WAR*']]
tt = tt.groupby('팀명').agg({"WAR*" : 'sum'}).reset_index()
tt

,팀명,WAR*
0,KIA,21.60
1,KT,14.02
2,LG,19.98
3,NC,22.06
4,SSG,16.59
5,두산,16.00
6,롯데,16.18
7,삼성,17.48
8,키움,17.95
9,한화,6.76


In [11]:
total_defense = hitter[['팀명', '실책']]
total_defense = total_defense.groupby('팀명').sum().reset_index()
total_defense

,팀명,실책
0,KIA,86
1,KT,76
2,LG,119
3,NC,107
4,SSG,102
5,두산,95
6,롯데,93
7,삼성,92
8,키움,105
9,한화,95


In [12]:
team_hit = hitter[['이름', '팀명', '포지션', '타석', '타수', '득점', '안타', '2루타', '3루타', '홈런', '루타', '타점', '볼넷', '사구', '삼진', '병살타', '희생플라이']]
team_run = hitter[['이름', '팀명', '포지션', '도루', '도루실패']]
team_defense = hitter[['이름', '팀명', '포지션', '실책']]
team_WAR = hitter[['이름', '팀명', '포지션', 'WAR*']]

In [13]:
team_hit = team_hit.groupby(['팀명', '포지션']).sum().reset_index()
team_hit.drop(columns='이름', inplace=True)
team_hit

,팀명,포지션,타석,타수,득점,안타,2루타,3루타,홈런,루타,타점,볼넷,사구,삼진,병살타,희생플라이
0,KIA,1루수,756,664,82,151,20,2,15,220,81,77,7,187,16,3.0
1,KIA,2루수,703,620,97,177,23,1,2,208,60,56,7,79,12,3.0
2,KIA,3루수,386,341,72,103,20,5,7,154,47,38,1,63,14,4.0
3,KIA,우익수,665,586,92,191,29,1,26,300,115,57,6,118,11,7.0
4,KIA,유격수,507,452,73,136,18,4,3,171,52,40,2,56,13,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,한화,유격수,698,615,69,139,26,2,3,178,33,48,12,135,15,2.0
85,한화,좌익수,682,619,72,140,28,4,13,215,77,42,9,176,13,7.0
86,한화,중견수,799,708,91,172,28,4,5,223,69,69,7,159,13,6.0
87,한화,지명타자,78,72,6,11,1,0,0,12,5,4,1,21,0,0.0


In [14]:
team_hit['타율'] = (team_hit['안타'] / team_hit['타수']).round(3)
team_hit['1루타'] = team_hit['안타'] - team_hit['2루타'] - team_hit['3루타'] - team_hit['홈런']
team_hit

,팀명,포지션,타석,타수,득점,안타,2루타,3루타,홈런,루타,타점,볼넷,사구,삼진,병살타,희생플라이,타율,1루타
0,KIA,1루수,756,664,82,151,20,2,15,220,81,77,7,187,16,3.0,0.227,114
1,KIA,2루수,703,620,97,177,23,1,2,208,60,56,7,79,12,3.0,0.285,151
2,KIA,3루수,386,341,72,103,20,5,7,154,47,38,1,63,14,4.0,0.302,71
3,KIA,우익수,665,586,92,191,29,1,26,300,115,57,6,118,11,7.0,0.326,135
4,KIA,유격수,507,452,73,136,18,4,3,171,52,40,2,56,13,6.0,0.301,111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,한화,유격수,698,615,69,139,26,2,3,178,33,48,12,135,15,2.0,0.226,108
85,한화,좌익수,682,619,72,140,28,4,13,215,77,42,9,176,13,7.0,0.226,95
86,한화,중견수,799,708,91,172,28,4,5,223,69,69,7,159,13,6.0,0.243,135
87,한화,지명타자,78,72,6,11,1,0,0,12,5,4,1,21,0,0.0,0.153,10


In [15]:
team_hit['장타율'] = ((team_hit['1루타'] + 2*team_hit['2루타'] + 3*team_hit['3루타'] + 4*team_hit['홈런']) / team_hit['타수']).round(3)
team_hit['출루율'] = ((team_hit['안타'] + team_hit['볼넷'] + team_hit['사구']) / (team_hit['타수'] + team_hit['볼넷'] + team_hit['사구'] + team_hit['희생플라이'])).round(3)
team_hit['효율'] = (team_hit['타점'] / team_hit['타수']).round(3)
team_hit

,팀명,포지션,타석,타수,득점,안타,2루타,3루타,홈런,루타,...,볼넷,사구,삼진,병살타,희생플라이,타율,1루타,장타율,출루율,효율
0,KIA,1루수,756,664,82,151,20,2,15,220,...,77,7,187,16,3.0,0.227,114,0.331,0.313,0.122
1,KIA,2루수,703,620,97,177,23,1,2,208,...,56,7,79,12,3.0,0.285,151,0.335,0.350,0.097
2,KIA,3루수,386,341,72,103,20,5,7,154,...,38,1,63,14,4.0,0.302,71,0.452,0.370,0.138
3,KIA,우익수,665,586,92,191,29,1,26,300,...,57,6,118,11,7.0,0.326,135,0.512,0.387,0.196
4,KIA,유격수,507,452,73,136,18,4,3,171,...,40,2,56,13,6.0,0.301,111,0.378,0.356,0.115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,한화,유격수,698,615,69,139,26,2,3,178,...,48,12,135,15,2.0,0.226,108,0.289,0.294,0.054
85,한화,좌익수,682,619,72,140,28,4,13,215,...,42,9,176,13,7.0,0.226,95,0.347,0.282,0.124
86,한화,중견수,799,708,91,172,28,4,5,223,...,69,7,159,13,6.0,0.243,135,0.315,0.314,0.097
87,한화,지명타자,78,72,6,11,1,0,0,12,...,4,1,21,0,0.0,0.153,10,0.167,0.208,0.069


In [16]:
# 주루는 생각해보니 포지션 상관없이 

team_run = team_run.groupby(['팀명']).sum().reset_index()
team_run.drop(columns=['이름', '포지션'], inplace=True)
team_run

,팀명,도루,도루실패
0,KIA,118,32
1,KT,87,30
2,LG,166,101
3,NC,111,46
4,SSG,96,28
5,두산,133,48
6,롯데,101,42
7,삼성,107,36
8,키움,54,12
9,한화,67,22


In [17]:
team_defense = team_defense.groupby(['팀명', '포지션']).sum().reset_index()
team_defense.drop(columns='이름', inplace=True)
team_defense

,팀명,포지션,실책
0,KIA,1루수,21
1,KIA,2루수,16
2,KIA,3루수,14
3,KIA,우익수,2
4,KIA,유격수,15
...,...,...,...
84,한화,유격수,23
85,한화,좌익수,8
86,한화,중견수,12
87,한화,지명타자,2


In [18]:
team_WAR = team_WAR.groupby(['팀명', '포지션']).agg({"WAR*" : 'sum'}).reset_index()
#team_WAR.drop(columns='이름', inplace=True)
team_WAR

,팀명,포지션,WAR*
0,KIA,1루수,0.63
1,KIA,2루수,2.79
2,KIA,3루수,3.71
3,KIA,우익수,5.65
4,KIA,유격수,3.69
...,...,...,...
84,한화,유격수,0.72
85,한화,좌익수,0.13
86,한화,중견수,1.34
87,한화,지명타자,-0.79


#### 투수

In [19]:
pitcher.columns

Index(['이름', 'id', '연도', '팀명', '평균자책점', '경기', '완투', '완봉', '승리', '패배', '세이브',
       '홀드', '승률', '타자수', '이닝', '피안타', '피홈런', '볼넷', '사구', '삼진', '실점', '자책점',
       '생년월일', '포지션', 'WAR', '폭투', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+',
       'WPA'],
      dtype='object')

In [20]:
pitcher

,이름,id,연도,팀명,평균자책점,경기,완투,완봉,승리,패배,...,생년월일,포지션,WAR,폭투,ERA,FIP,WHIP,ERA+,FIP+,WPA
0,유영찬,50106,2023,LG,3.44,67,0,0,6,3,...,1997-03-07,중계,1.55,8.0,3.44,4.58,1.40,118.6,89.5,-0.12
1,이민호,50126,2023,LG,5.03,5,0,0,0,2,...,2001-08-30,선발,-0.20,0.0,5.03,4.92,1.63,81.1,83.4,-0.43
2,김윤식,50157,2023,LG,4.22,17,0,0,6,4,...,2000-04-03,선발,1.00,4.0,4.22,4.14,1.49,96.7,98.9,0.40
3,이종민,50354,2023,키움,7.24,11,0,0,0,1,...,2001-06-04,중계,-0.36,0.0,7.24,3.52,2.05,56.7,117.1,-0.71
4,김동혁,50360,2023,키움,7.32,35,0,0,1,7,...,2001-12-27,선발,-0.81,0.0,7.55,4.38,1.75,54.4,93.7,-1.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,장필준,77927,2023,삼성,7.91,17,0,0,1,1,...,1988-04-08,선발,-0.30,0.0,7.91,5.82,2.07,53.5,71.8,-0.34
197,정찬헌,78148,2023,키움,4.75,14,1,0,2,8,...,1990-01-26,선발,0.67,0.0,4.75,3.78,1.17,86.5,108.1,-0.15
198,임창민,78352,2023,키움,2.51,51,0,0,2,2,...,1985-08-25,중계,1.89,0.0,2.51,3.59,1.46,163.8,113.1,0.55
199,하준호,78517,2023,KT,4.15,12,0,0,0,1,...,1989-04-29,중계,0.04,4.0,4.85,5.52,1.85,87.1,75.5,-0.15


In [21]:
team_pitch = pitcher[['이름', '팀명', '포지션', '경기', '완투', '완봉', '승리', '패배', '세이브',
       '홀드', '타자수', '이닝', '피안타', '피홈런', '볼넷', '사구', '삼진', '실점', '자책점', 'WPA']]
team_WAR_p = pitcher[['이름', '팀명', '포지션', 'WAR']]

#FIP : 수비의 영향을 받지 않고 투수 평가
#WHIP : (볼넷 수 + 안타 수)/ 던진 이닝 수 : 주자를 얼마나 많이 내보냈는가?
#WPA : 투수의 각 플레이가 팀의 승리 확률에 얼마나 기여했는지?

In [107]:
pitcher.columns

Index(['이름', 'id', '연도', '팀명', '평균자책점', '경기', '완투', '완봉', '승리', '패배', '세이브',
       '홀드', '승률', '타자수', '이닝', '피안타', '피홈런', '볼넷', '사구', '삼진', '실점', '자책점',
       '생년월일', '포지션', 'WAR', '폭투', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+',
       'WPA'],
      dtype='object')

In [110]:
total_pitch = pitcher.groupby('팀명').agg({
    '피홈런': 'sum',    # 리그 전체 홈런 수
    '볼넷': 'sum',    # 리그 전체 볼넷 수
    '사구': 'sum',    # 리그 전체 사구 수
    '삼진': 'sum',  # 리그 전체 삼진 수
    '이닝': 'sum',   # 리그 전체 이닝 수
    '자책점': 'sum'  # 리그 전체 자책점 수
}).reset_index()
total_pitch['FIP'] = ((13 * total_pitch['피홈런']) + (3 * (total_pitch['볼넷'] + total_pitch['사구'])) - (2 * total_pitch['삼진'])) / total_pitch['이닝'] + 3.37
total_pitch

,팀명,피홈런,볼넷,사구,삼진,이닝,자책점,FIP
0,KIA,86,530,56,950,1232.0,550,4.162208
1,KT,78,373,44,913,1217.1,513,3.730693
2,LG,75,470,76,994,1303.5,524,3.849478
3,NC,95,477,58,1023,1207.4,504,4.027611
4,SSG,99,583,63,949,1254.7,602,4.427623
5,두산,83,423,73,940,1188.5,495,3.948040
6,롯데,76,501,79,1047,1231.7,562,3.884736
7,삼성,117,473,52,943,1320.3,676,4.286458
8,키움,71,463,51,853,1130.5,552,4.041384
9,한화,93,463,91,978,1227.1,557,4.115661


In [23]:
league_era = 5535 / (12312.8 / 9)

# 리그 평균 FIP 계산 (FIP_constant를 계산하기 위한 중간 단계)
league_fip = ((13 * 873) + (3 * (4756 + 643)) - 
              (2 * 9590)) / 12312.8

# FIP_constant 계산
fip_constant = league_era - league_fip

# 결과 출력
fip_constant

3.366334221298162

In [102]:
ta = team_pitch.groupby('팀명').sum().reset_index()
ta['팀명'] = ta['팀명'].apply(lambda x: '한화' if x == '한화' else '타 팀')
ta = ta.groupby('팀명').sum().reset_index()
ta['FIP'] = ((13 * ta['피홈런']) + (3 * (ta['볼넷'] + ta['사구'])) - (2 * ta['삼진'])) / ta['이닝'] + 3.37
ta

,팀명,포지션,경기,완투,완봉,승리,패배,세이브,홀드,타자수,...,볼넷,사구,삼진,실점,자책점,WPA,승률,평균자책점,FIP,WHIP
0,타 팀,선발중계선발중계선발중계선발중계선발중계선발중계선발중계선발중계선발중계,5652,5,0,650,628,331,648,48293,...,4293,552,8612,5567,4978,-0.13,9.177,73.17,4.042127,25.343751
1,한화,선발중계,694,0,0,58,80,19,68,5356,...,463,91,978,634,557,-4.31,0.908,7.96,4.115661,2.707523


In [24]:
team_pitch = team_pitch.groupby(['팀명', '포지션']).sum().reset_index()
team_pitch.drop(columns='이름', inplace=True)
team_pitch['승률'] = ((team_pitch['승리']) / (team_pitch['승리'] + team_pitch['패배'])).round(3)
team_pitch['평균자책점'] = (((team_pitch['자책점']) / (team_pitch['이닝'])) * 9).round(2)
team_pitch['FIP'] = ((13 * team_pitch['피홈런']) + (3 * (team_pitch['볼넷'] + team_pitch['사구'])) - (2 * team_pitch['삼진'])) / team_pitch['이닝'] + 3.37
team_pitch['WHIP'] = (team_pitch['볼넷'] + team_pitch['피안타']) / team_pitch['이닝']

team_pitch

,팀명,포지션,경기,완투,완봉,승리,패배,세이브,홀드,타자수,...,볼넷,사구,삼진,실점,자책점,WPA,승률,평균자책점,FIP,WHIP
0,KIA,선발,155,1,0,44,49,0,0,3315,...,307,29,616,409,369,-3.15,0.473,4.39,4.054134,1.419876
1,KIA,중계,519,0,0,29,20,33,66,2051,...,223,27,334,207,181,0.31,0.592,3.42,4.333678,1.377283
2,KT,선발,228,0,0,59,43,0,6,3810,...,258,30,654,415,380,2.56,0.578,3.85,3.689460,1.338583
3,KT,중계,309,0,0,20,19,37,54,1392,...,115,14,259,147,133,4.54,0.513,3.65,3.842417,1.289241
4,LG,선발,204,0,0,61,36,3,4,3815,...,278,40,611,428,374,0.25,0.629,3.81,3.831591,1.339518
5,LG,중계,446,0,0,31,24,32,85,1790,...,192,36,383,168,150,3.03,0.564,3.22,3.887159,1.272641
6,NC,선발,177,0,0,48,39,0,0,3135,...,257,34,611,341,301,2.23,0.552,3.68,4.043273,1.256957
7,NC,중계,488,0,0,23,26,33,88,2032,...,220,24,412,228,203,0.60,0.469,3.88,4.003100,1.368175
8,SSG,선발,216,1,0,46,49,1,12,3737,...,364,47,608,471,425,-2.75,0.484,4.49,4.519601,1.480742
9,SSG,중계,389,0,0,30,16,45,59,1798,...,219,16,341,192,177,3.80,0.652,3.95,4.233309,1.552965


In [103]:
total_pitch

,팀명,실점,경기,완투,완봉,승리,패배,세이브,홀드,타자수,이닝,피안타,피홈런,볼넷,사구,삼진,자책점,승률,평균자책점
2,LG,596,650,0,0,92,60,35,89,5605,1303.5,1248,75,470,76,994,524,0.605,3.62
3,NC,569,665,0,0,71,65,33,88,5167,1207.4,1093,95,477,58,1023,504,0.522,3.76
6,롯데,635,649,1,0,68,76,35,81,5466,1231.7,1292,76,501,79,1047,562,0.472,4.11
7,삼성,748,690,1,0,66,84,38,72,5842,1320.3,1447,117,473,52,943,676,0.440,4.61
4,SSG,663,605,1,0,76,65,46,71,5535,1254.7,1304,99,583,63,949,602,0.539,4.32
9,한화,634,694,0,0,58,80,19,68,5356,1227.1,1217,93,463,91,978,557,0.420,4.09
0,KIA,616,674,1,0,73,69,33,66,5366,1232.0,1199,86,530,56,950,550,0.514,4.02
5,두산,551,567,0,0,74,68,41,65,5098,1188.5,1133,83,423,73,940,495,0.521,3.75
1,KT,562,537,0,0,79,62,37,60,5202,1217.1,1240,78,373,44,913,513,0.560,3.79
8,키움,627,615,1,0,51,79,33,56,5012,1130.5,1186,71,463,51,853,552,0.392,4.39


In [25]:
team_WAR_p = team_WAR_p.groupby(['팀명', '포지션']).agg({"WAR" : 'sum'}).reset_index()
#team_WAR_p.drop(columns='이름', inplace=True)
team_WAR_p

,팀명,포지션,WAR
0,KIA,선발,5.81
1,KIA,중계,10.21
2,KT,선발,16.08
3,KT,중계,8.19
4,LG,선발,8.50
5,LG,중계,9.97
6,NC,선발,13.18
7,NC,중계,7.64
8,SSG,선발,7.69
9,SSG,중계,8.02


#### 시각화

In [26]:
team_colors = {
    "키움": "#F7CAC9",  # 키움: 연한 보라색
    "두산": "#FFF0AC",  # 두산: 연한 노랑색
    "롯데": "#A4DDED",  # 롯데: 연한 파랑색
    "삼성": "#92A8D1",  # 삼성: 연한 남색
    "한화": "#F9AA8F",  # 한화: 연한 주황색
    "KIA": "#F7786B",  # KIA: 연한 빨강색
    "LG": "#D6AEDD",  # LG: 연한 자주색
    "SSG": "#FA9A85",  # SSG: 연한 붉은색
    "NC": "#B6E2D3",  # NC: 연한 녹색
    "KT": "#B0A8B9"   # KT: 연한 회색
}

In [27]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [28]:
def plot_positional_comparison(df, metric_column, team_name):
    """
    주어진 DataFrame에서 지정된 지표를 기준으로 각 포지션별로 데이터를 높은 순서대로 정렬하여 그래프를 생성하고,
    특정 팀의 순위를 표시합니다.

    :param df: DataFrame
    :param metric_column: 비교할 지표의 컬럼명
    :param team_name: 순위를 표시할 팀명
    """
    positions = df['포지션'].unique()


    # Subplots 생성
    fig = make_subplots(rows=len(positions), cols=1, shared_xaxes=True)

    for i, position in enumerate(positions, 1):
        # 데이터를 지표에 따라 정렬합니다.
        sorted_df = df[df['포지션'] == position].sort_values(by=metric_column, ascending=False)
        sorted_df['color'] = sorted_df['팀명'].map(team_colors)

        # 특정 팀의 순위를 찾아 표시합니다.
        team_metric = sorted_df[sorted_df['팀명'] == team_name][metric_column].values[0]
        team_rank = sorted_df['팀명'].tolist().index(team_name) + 1
        
        rank_y_position = team_metric + (df[metric_column].max() - df[metric_column].min()) * 0.1

        # 각 포지션별로 Bar 그래프를 추가합니다.
        fig.add_trace(
            go.Bar(
                x=sorted_df['팀명'],
                y=sorted_df[metric_column],
                name=position,
                marker_color=sorted_df['color'],
                text=sorted_df[metric_column],  # 막대 위에 값 표시
                textposition='auto',
                texttemplate='%{text:.2f}'
            ),
            row=i,
            col=1
        )
        
        # 특정 팀의 막대에 순위 표시
        fig.add_annotation(
            x=team_name,
            y=rank_y_position * 1.05,  # 수정된 높이 위치
            text=f'{team_rank}등',
            showarrow=False,
            row=i,
            col=1
        )

    # 레이아웃 설정
    fig.update_layout(
        title_text=f'포지션별 {metric_column} (높은 순서대로)',
        showlegend=False,
        height=300 * len(positions),
        width = 1000
    )
    
    fig.show()


In [29]:
plot_positional_comparison(team_hit, '타율', '한화')

In [30]:
total_hit = hitter[['팀명', '타석', '타수', '득점', '안타', '2루타', '3루타', '도루', '홈런', '루타', '타점', '볼넷', '사구', '삼진', '병살타', '희생플라이']]
total_hit = total_hit.groupby('팀명').sum().reset_index()
total_hit['타율'] = (total_hit['안타'] / total_hit['타수']).round(3)
total_hit['1루타'] = total_hit['안타'] - total_hit['2루타'] - total_hit['3루타'] - total_hit['홈런']
total_hit['장타율'] = ((total_hit['1루타'] + 2*total_hit['2루타'] + 3*total_hit['3루타'] + 4*total_hit['홈런']) / total_hit['타수']).round(3)
total_hit['출루율'] = ((total_hit['안타'] + total_hit['볼넷'] + total_hit['사구']) / (total_hit['타수'] + total_hit['볼넷'] + total_hit['사구'] + total_hit['희생플라이'])).round(3)
total_hit['효율'] = (total_hit['타점'] / team_hit['타수']).round(3)
total_hit

,팀명,타석,타수,득점,안타,2루타,3루타,도루,홈런,루타,...,볼넷,사구,삼진,병살타,희생플라이,타율,1루타,장타율,출루율,효율
0,KIA,5497,4859,705,1338,222,21,118,102,1908,...,483,47,925,118,43.0,0.275,993,0.393,0.344,1.015
1,KT,5648,4973,672,1316,235,14,87,89,1846,...,523,48,1074,96,47.0,0.265,978,0.371,0.338,1.002
2,LG,5671,4866,765,1360,225,27,166,93,1918,...,583,72,799,89,55.0,0.279,1015,0.394,0.361,2.088
3,NC,5605,4894,679,1321,223,28,111,98,1894,...,493,100,1005,122,55.0,0.270,972,0.387,0.345,1.096
4,SSG,5633,4926,657,1280,229,16,96,125,1916,...,507,78,943,89,45.0,0.260,910,0.389,0.336,1.352
5,두산,5535,4856,620,1237,210,32,133,100,1811,...,505,76,975,90,41.0,0.255,895,0.373,0.332,1.099
6,롯데,5575,4870,653,1289,231,19,101,69,1765,...,522,51,999,110,57.0,0.265,970,0.362,0.339,0.947
7,삼성,5618,4948,651,1296,206,23,107,86,1806,...,490,61,978,110,42.0,0.262,981,0.365,0.333,1.374
8,키움,5817,5135,615,1348,229,30,54,62,1823,...,498,75,1109,88,58.0,0.263,1027,0.355,0.333,0.966
9,한화,5628,4906,604,1184,211,12,67,100,1719,...,535,88,1162,103,40.0,0.241,861,0.350,0.324,0.902


In [31]:
total_pitch =  pitcher[['팀명', '실점', '경기', '완투', '완봉', '승리', '패배', '세이브','홀드', '타자수', '이닝', '피안타', '피홈런', '볼넷', '사구', '삼진', '자책점']]
total_pitch = total_pitch.groupby('팀명').sum().reset_index()
total_pitch['승률'] = ((total_pitch['승리']) / (total_pitch['승리'] + total_pitch['패배'])).round(3)
total_pitch['평균자책점'] = (((total_pitch['자책점']) / (total_pitch['이닝'])) * 9).round(2)
total_pitch

,팀명,실점,경기,완투,완봉,승리,패배,세이브,홀드,타자수,이닝,피안타,피홈런,볼넷,사구,삼진,자책점,승률,평균자책점
0,KIA,616,674,1,0,73,69,33,66,5366,1232.0,1199,86,530,56,950,550,0.514,4.02
1,KT,562,537,0,0,79,62,37,60,5202,1217.1,1240,78,373,44,913,513,0.560,3.79
2,LG,596,650,0,0,92,60,35,89,5605,1303.5,1248,75,470,76,994,524,0.605,3.62
3,NC,569,665,0,0,71,65,33,88,5167,1207.4,1093,95,477,58,1023,504,0.522,3.76
4,SSG,663,605,1,0,76,65,46,71,5535,1254.7,1304,99,583,63,949,602,0.539,4.32
5,두산,551,567,0,0,74,68,41,65,5098,1188.5,1133,83,423,73,940,495,0.521,3.75
6,롯데,635,649,1,0,68,76,35,81,5466,1231.7,1292,76,501,79,1047,562,0.472,4.11
7,삼성,748,690,1,0,66,84,38,72,5842,1320.3,1447,117,473,52,943,676,0.440,4.61
8,키움,627,615,1,0,51,79,33,56,5012,1130.5,1186,71,463,51,853,552,0.392,4.39
9,한화,634,694,0,0,58,80,19,68,5356,1227.1,1217,93,463,91,978,557,0.420,4.09


In [32]:
fig = go.Figure()

total_hit = total_hit.sort_values(by='득점', ascending=False)

# 막대 추가
for index, row in total_hit.iterrows():
    fig.add_trace(go.Bar(
        x=[row['팀명']],
        y=[row['득점']],
        name=row['팀명'],
        marker=dict(color=team_colors[row['팀명']]),# 승률에 따라 색상을 다르게 설정
    ))

# 막대 간격을 넓히기 위해 레이아웃을 조정
fig.update_layout(
    barmode='relative',
    bargap=0.5,  # 막대 간격 설정
    title_text='팀별 득점 비교',  # 그래프 제목
    xaxis=dict(title='팀명'),  # x축 제목
    yaxis=dict(title='득점'),  # y축 제목
    showlegend=True,
    width = 1000
)

# 그래프 보여주기
fig.show()

In [33]:
total_hit

,팀명,타석,타수,득점,안타,2루타,3루타,도루,홈런,루타,...,볼넷,사구,삼진,병살타,희생플라이,타율,1루타,장타율,출루율,효율
2,LG,5671,4866,765,1360,225,27,166,93,1918,...,583,72,799,89,55.0,0.279,1015,0.394,0.361,2.088
0,KIA,5497,4859,705,1338,222,21,118,102,1908,...,483,47,925,118,43.0,0.275,993,0.393,0.344,1.015
3,NC,5605,4894,679,1321,223,28,111,98,1894,...,493,100,1005,122,55.0,0.270,972,0.387,0.345,1.096
1,KT,5648,4973,672,1316,235,14,87,89,1846,...,523,48,1074,96,47.0,0.265,978,0.371,0.338,1.002
4,SSG,5633,4926,657,1280,229,16,96,125,1916,...,507,78,943,89,45.0,0.260,910,0.389,0.336,1.352
6,롯데,5575,4870,653,1289,231,19,101,69,1765,...,522,51,999,110,57.0,0.265,970,0.362,0.339,0.947
7,삼성,5618,4948,651,1296,206,23,107,86,1806,...,490,61,978,110,42.0,0.262,981,0.365,0.333,1.374
5,두산,5535,4856,620,1237,210,32,133,100,1811,...,505,76,975,90,41.0,0.255,895,0.373,0.332,1.099
8,키움,5817,5135,615,1348,229,30,54,62,1823,...,498,75,1109,88,58.0,0.263,1027,0.355,0.333,0.966
9,한화,5628,4906,604,1184,211,12,67,100,1719,...,535,88,1162,103,40.0,0.241,861,0.350,0.324,0.902


In [34]:
fig = go.Figure()

total_hit = total_hit.sort_values(by='도루', ascending=False)

# 막대 추가
for index, row in total_hit.iterrows():
    fig.add_trace(go.Bar(
        x=[row['팀명']],
        y=[row['도루']],
        name=row['팀명'],
        marker=dict(color=team_colors[row['팀명']]),# 승률에 따라 색상을 다르게 설정
    ))

# 막대 간격을 넓히기 위해 레이아웃을 조정
fig.update_layout(
    barmode='relative',
    bargap=0.5,  # 막대 간격 설정
    title_text='팀별 도루 횟수 비교',  # 그래프 제목
    xaxis=dict(title='팀명'),  # x축 제목
    yaxis=dict(title='도루'),  # y축 제목
    showlegend=True,
    width = 1000
)

# 그래프 보여주기
fig.show()

In [35]:
fig = go.Figure()

total_pitch = total_pitch.sort_values(by='실점', ascending=False)

# 막대 추가
for index, row in total_pitch.iterrows():
    fig.add_trace(go.Bar(
        x=[row['팀명']],
        y=[row['실점']],
        name=row['팀명'],
        marker=dict(color=team_colors[row['팀명']]),# 승률에 따라 색상을 다르게 설정
    ))

# 막대 간격을 넓히기 위해 레이아웃을 조정
fig.update_layout(
    barmode='relative',
    bargap=0.5,  # 막대 간격 설정
    title_text='팀별 실점 비교',  # 그래프 제목
    xaxis=dict(title='팀명'),  # x축 제목
    yaxis=dict(title='실점'),  # y축 제목
    showlegend=True,
    width = 1000
)

# 그래프 보여주기
fig.show()

In [36]:
total_defense = total_defense.sort_values(by='실책', ascending=False)
total_defense

,팀명,실책
2,LG,119
3,NC,107
8,키움,105
4,SSG,102
5,두산,95
9,한화,95
6,롯데,93
7,삼성,92
0,KIA,86
1,KT,76


In [37]:
fig = go.Figure()

total_defense = total_defense.sort_values(by='실책', ascending=False)

# 막대 추가
for index, row in total_defense.iterrows():
    fig.add_trace(go.Bar(
        x=[row['팀명']],
        y=[row['실책']],
        name=row['팀명'],
        marker=dict(color=team_colors[row['팀명']]),# 승률에 따라 색상을 다르게 설정
    ))

# 막대 간격을 넓히기 위해 레이아웃을 조정
fig.update_layout(
    barmode='relative',
    bargap=0.5,  # 막대 간격 설정
    title_text='팀별 실책 비교',  # 그래프 제목
    xaxis=dict(title='팀명'),  # x축 제목
    yaxis=dict(title='실책'),  # y축 제목
    showlegend=True,
    width = 1000
)

# 그래프 보여주기
fig.show()

In [38]:
fig = go.Figure()

total_pitch = total_pitch.sort_values(by='홀드', ascending=False)

# 막대 추가
for index, row in total_pitch.iterrows():
    fig.add_trace(go.Bar(
        x=[row['팀명']],
        y=[row['홀드']],
        name=row['팀명'],
        marker=dict(color=team_colors[row['팀명']]),# 승률에 따라 색상을 다르게 설정
    ))

# 막대 간격을 넓히기 위해 레이아웃을 조정
fig.update_layout(
    barmode='relative',
    bargap=0.5,  # 막대 간격 설정
    title_text='팀별 홀드 비교',  # 그래프 제목
    xaxis=dict(title='팀명'),  # x축 제목
    yaxis=dict(title='홀드'),  # y축 제목
    showlegend=True,
    width = 1000
)

# 그래프 보여주기
fig.show()

In [39]:
fig = go.Figure()

total_pitch = total_pitch.sort_values(by='세이브', ascending=False)

# 막대 추가
for index, row in total_pitch.iterrows():
    fig.add_trace(go.Bar(
        x=[row['팀명']],
        y=[row['세이브']],
        name=row['팀명'],
        marker=dict(color=team_colors[row['팀명']]),# 승률에 따라 색상을 다르게 설정
    ))

# 막대 간격을 넓히기 위해 레이아웃을 조정
fig.update_layout(
    barmode='relative',
    bargap=0.5,  # 막대 간격 설정
    title_text='팀별 세이브 비교',  # 그래프 제목
    xaxis=dict(title='팀명'),  # x축 제목
    yaxis=dict(title='세이브'),  # y축 제목
    showlegend=True,
    width = 1000
)

# 그래프 보여주기
fig.show()

In [40]:
fig = go.Figure()

total_pitch = total_pitch.sort_values(by='평균자책점', ascending=False)

# 막대 추가
for index, row in total_pitch.iterrows():
    fig.add_trace(go.Bar(
        x=[row['팀명']],
        y=[row['평균자책점']],
        name=row['팀명'],
        marker=dict(color=team_colors[row['팀명']]),# 승률에 따라 색상을 다르게 설정
    ))

# 막대 간격을 넓히기 위해 레이아웃을 조정
fig.update_layout(
    barmode='relative',
    bargap=0.5,  # 막대 간격 설정
    title_text='팀별 평균자책점 비교',  # 그래프 제목
    xaxis=dict(title='팀명'),  # x축 제목
    yaxis=dict(title='평균자책점'),  # y축 제목
    showlegend=True,
    width = 1000
)

# 그래프 보여주기
fig.show()

In [41]:
temp = pitcher.loc[(pitcher['이닝'] > 30) & (pitcher['ERA+'] > 100), ['팀명', '이름', 'ERA+']]
temp = temp.groupby('팀명').count().reset_index()
temp

,팀명,이름,ERA+
0,KIA,10,10
1,KT,8,8
2,LG,10,10
3,NC,8,8
4,SSG,6,6
5,두산,8,8
6,롯데,9,9
7,삼성,7,7
8,키움,4,4
9,한화,9,9


In [42]:
fig = go.Figure()


temp = temp.sort_values('ERA+', ascending=False)

# 막대 추가
for index, row in temp.iterrows():
    fig.add_trace(go.Bar(
        x=[row['팀명']],
        y=[row['ERA+']],
        name=row['팀명'],
        marker=dict(color=team_colors[row['팀명']]),# 승률에 따라 색상을 다르게 설정
    ))

# 막대 간격을 넓히기 위해 레이아웃을 조정
fig.update_layout(
    barmode='relative',
    bargap=0.5,  # 막대 간격 설정
    title_text='팀별 ERA+ 100이상 선수 명 수',  # 그래프 제목
    xaxis=dict(title='팀명'),  # x축 제목
    yaxis=dict(title='ERA+'),  # y축 제목
    showlegend=True,
    width = 1000
)

# 그래프 보여주기
fig.show()

In [84]:
fig = go.Figure()

total_pitch = total_pitch.sort_values(by='세이브', ascending=False)

# 막대 추가
for index, row in total_pitch.iterrows():
    fig.add_trace(go.Bar(
        x=[row['팀명']],
        y=[row['세이브']],
        name=row['팀명'],
        marker=dict(color=team_colors[row['팀명']]),# 승률에 따라 색상을 다르게 설정
    ))

# 막대 간격을 넓히기 위해 레이아웃을 조정
fig.update_layout(
    barmode='relative',
    bargap=0.5,  # 막대 간격 설정
    title_text='팀별 세이브 비교',  # 그래프 제목
    xaxis=dict(title='팀명'),  # x축 제목
    yaxis=dict(title='세이브'),  # y축 제목
    showlegend=True,
    width = 1000
)

# 그래프 보여주기
fig.show()

In [123]:
team_colors = {'키움': '#F7CAC9',
 '두산': '#FFF0AC',
 '롯데': '#A4DDED',
 '삼성': '#92A8D1',
 '한화': '#F9AA8F',
 'KIA': '#F7786B',
 'LG': '#D6AEDD',
 'SSG': '#FA9A85',
 'NC': '#B6E2D3',
 'KT': '#B0A8B9',
 '타 팀' : '#D6AEDD'}

In [124]:
fig = go.Figure()

temp = total_pitch.sort_values(by='FIP', ascending = True)

# 막대 추가
for index, row in temp.iterrows():
    fig.add_trace(go.Bar(
        x=[row['팀명']],
        y=[row['FIP']],
        name=row['팀명'],
        text=row['FIP'],
        textposition='auto',
        texttemplate='%{text:.2f}',
        marker=dict(color=team_colors[row['팀명']]),# 승률에 따라 색상을 다르게 설정
    ))

# 막대 간격을 넓히기 위해 레이아웃을 조정
fig.update_layout(
    barmode='relative',
    bargap=0.5,  # 막대 간격 설정
    title_text='팀별 FIP 비교',  # 그래프 제목
    xaxis=dict(title='팀'),  # x축 제목
    yaxis=dict(title='FIP'),  # y축 제목
    showlegend=True,
    width = 1000
)

# 그래프 보여주기
fig.show()

In [140]:
team_pitch.loc[team_pitch['포지션'] == '선발', ['팀명', '이닝', '승리', '패배', '볼넷', '사구', '삼진', '실점', '자책점', 'FIP', '평균자책점']]

,팀명,이닝,승리,패배,볼넷,사구,삼진,실점,자책점,FIP,평균자책점
0,KIA,755.7,44,49,307,29,616,409,369,4.054134,4.39
2,KT,889.0,59,43,258,30,654,415,380,3.689460,3.85
4,LG,883.9,61,36,278,40,611,428,374,3.831591,3.81
6,NC,736.7,48,39,257,34,611,341,301,4.043273,3.68
8,SSG,851.6,46,49,364,47,608,471,425,4.519601,4.49
10,두산,771.0,48,47,236,44,606,332,303,3.865460,3.54
12,롯데,821.6,46,50,301,54,669,408,355,3.797215,3.89
14,삼성,949.8,43,57,299,35,675,501,445,4.030139,4.22
16,키움,759.1,36,56,286,39,590,400,362,3.819216,4.29
18,한화,844.0,36,61,294,60,669,454,399,4.136588,4.25


In [156]:
fig = go.Figure()

temp = team_pitch.copy()
temp = temp.loc[temp['이닝'] > 30]
temp['팀명'] = temp['팀명'].apply(lambda x: '한화' if x == '한화' else '타 팀')
temp = temp.groupby(['팀명', '포지션']).mean().reset_index()
temp = temp.loc[temp['포지션'] == '선발']
# 막대 추가
for index, row in temp.iterrows():
    fig.add_trace(go.Bar(
        x=[row['팀명']],
        y=[row['패배']],
        name=row['팀명'],
        text=row['패배'],
        textposition='auto',
        texttemplate='%{text:.1f}',
        marker=dict(color=team_colors[row['팀명']]),# 승률에 따라 색상을 다르게 설정
    ))

# 막대 간격을 넓히기 위해 레이아웃을 조정
fig.update_layout(
    barmode='relative',
    bargap=0.5,  # 막대 간격 설정
    title_text='팀별 패배 비교',  # 그래프 제목
    xaxis=dict(title='팀명'),  # x축 제목
    yaxis=dict(title='패배'),  # y축 제목
    showlegend=True,
    width = 400
)

# 그래프 보여주기
fig.show()

In [163]:
team_pitch

,팀명,포지션,경기,완투,완봉,승리,패배,세이브,홀드,타자수,...,볼넷,사구,삼진,실점,자책점,WPA,승률,평균자책점,FIP,WHIP
0,KIA,선발,155,1,0,44,49,0,0,3315,...,307,29,616,409,369,-3.15,0.473,4.39,4.054134,1.419876
1,KIA,중계,519,0,0,29,20,33,66,2051,...,223,27,334,207,181,0.31,0.592,3.42,4.333678,1.377283
2,KT,선발,228,0,0,59,43,0,6,3810,...,258,30,654,415,380,2.56,0.578,3.85,3.689460,1.338583
3,KT,중계,309,0,0,20,19,37,54,1392,...,115,14,259,147,133,4.54,0.513,3.65,3.842417,1.289241
4,LG,선발,204,0,0,61,36,3,4,3815,...,278,40,611,428,374,0.25,0.629,3.81,3.831591,1.339518
5,LG,중계,446,0,0,31,24,32,85,1790,...,192,36,383,168,150,3.03,0.564,3.22,3.887159,1.272641
6,NC,선발,177,0,0,48,39,0,0,3135,...,257,34,611,341,301,2.23,0.552,3.68,4.043273,1.256957
7,NC,중계,488,0,0,23,26,33,88,2032,...,220,24,412,228,203,0.60,0.469,3.88,4.003100,1.368175
8,SSG,선발,216,1,0,46,49,1,12,3737,...,364,47,608,471,425,-2.75,0.484,4.49,4.519601,1.480742
9,SSG,중계,389,0,0,30,16,45,59,1798,...,219,16,341,192,177,3.80,0.652,3.95,4.233309,1.552965


In [164]:
fig = go.Figure()

temp = team_pitch.copy()
temp = temp.loc[temp['이닝'] > 30]
temp['팀명'] = temp['팀명'].apply(lambda x: '한화' if x == '한화' else '타 팀')
temp = temp.groupby(['팀명', '포지션']).mean().reset_index()
temp = temp.loc[temp['포지션'] == '선발']
# 막대 추가
for index, row in temp.iterrows():
    fig.add_trace(go.Bar(
        x=[row['팀명']],
        y=[row['실점']],
        name=row['팀명'],
        text=row['실점']/row['이닝'],
        textposition='auto',
        texttemplate='%{text:.2f}',
        marker=dict(color=team_colors[row['팀명']]),# 승률에 따라 색상을 다르게 설정
    ))

# 막대 간격을 넓히기 위해 레이아웃을 조정
fig.update_layout(
    barmode='relative',
    bargap=0.5,  # 막대 간격 설정
    title_text='팀별 실점 비교',  # 그래프 제목
    xaxis=dict(title='팀명'),  # x축 제목
    yaxis=dict(title='이닝 대비 실점'),  # y축 제목
    showlegend=True,
    width = 400
)

# 그래프 보여주기
fig.show()

In [157]:
fig = go.Figure()

temp = team_pitch.copy()
temp = temp.loc[temp['이닝'] > 30]
temp['팀명'] = temp['팀명'].apply(lambda x: '한화' if x == '한화' else '타 팀')
temp = temp.groupby(['팀명', '포지션']).mean().reset_index()
temp = temp.loc[temp['포지션'] == '선발']
# 막대 추가
for index, row in temp.iterrows():
    fig.add_trace(go.Bar(
        x=[row['팀명']],
        y=[row['FIP']],
        name=row['팀명'],
        text=row['FIP'],
        textposition='auto',
        texttemplate='%{text:.2f}',
        marker=dict(color=team_colors[row['팀명']]),# 승률에 따라 색상을 다르게 설정
    ))

# 막대 간격을 넓히기 위해 레이아웃을 조정
fig.update_layout(
    barmode='relative',
    bargap=0.5,  # 막대 간격 설정
    title_text='팀별 FIP 비교',  # 그래프 제목
    xaxis=dict(title='팀명'),  # x축 제목
    yaxis=dict(title='FIP'),  # y축 제목
    showlegend=True,
    width = 400
)

# 그래프 보여주기
fig.show()

In [158]:
team_pitch

,팀명,포지션,경기,완투,완봉,승리,패배,세이브,홀드,타자수,...,볼넷,사구,삼진,실점,자책점,WPA,승률,평균자책점,FIP,WHIP
0,KIA,선발,155,1,0,44,49,0,0,3315,...,307,29,616,409,369,-3.15,0.473,4.39,4.054134,1.419876
1,KIA,중계,519,0,0,29,20,33,66,2051,...,223,27,334,207,181,0.31,0.592,3.42,4.333678,1.377283
2,KT,선발,228,0,0,59,43,0,6,3810,...,258,30,654,415,380,2.56,0.578,3.85,3.689460,1.338583
3,KT,중계,309,0,0,20,19,37,54,1392,...,115,14,259,147,133,4.54,0.513,3.65,3.842417,1.289241
4,LG,선발,204,0,0,61,36,3,4,3815,...,278,40,611,428,374,0.25,0.629,3.81,3.831591,1.339518
5,LG,중계,446,0,0,31,24,32,85,1790,...,192,36,383,168,150,3.03,0.564,3.22,3.887159,1.272641
6,NC,선발,177,0,0,48,39,0,0,3135,...,257,34,611,341,301,2.23,0.552,3.68,4.043273,1.256957
7,NC,중계,488,0,0,23,26,33,88,2032,...,220,24,412,228,203,0.60,0.469,3.88,4.003100,1.368175
8,SSG,선발,216,1,0,46,49,1,12,3737,...,364,47,608,471,425,-2.75,0.484,4.49,4.519601,1.480742
9,SSG,중계,389,0,0,30,16,45,59,1798,...,219,16,341,192,177,3.80,0.652,3.95,4.233309,1.552965


In [166]:
pitcher.loc[(pitcher['이닝'] > 40) & (pitcher['포지션'] == '중계') & (pitcher['팀명'] == '한화'), ['팀명', '이름', 'WAR', 'FIP']].sort_values('WAR', ascending=False)

,팀명,이름,WAR,FIP
120,한화,주현상,2.58,3.22
92,한화,윤대경,1.11,4.93
122,한화,김범수,0.77,3.89
140,한화,박상원,0.37,3.38
184,한화,정우람,0.24,4.34
11,한화,강재민,-0.37,4.87


In [132]:
fig = go.Figure()

temp = team_pitch.copy()
temp['팀명'] = temp['팀명'].apply(lambda x: '한화' if x == '한화' else '타 팀')
temp = temp.groupby(['팀명', '포지션']).mean().reset_index()
temp = temp.loc[temp['포지션'] == '중계']
# 막대 추가
for index, row in temp.iterrows():
    fig.add_trace(go.Bar(
        x=[row['팀명']],
        y=[row['패배']],
        name=row['팀명'],
        text=row['패배'],
        textposition='auto',
        texttemplate='%{text:.1f}',
        marker=dict(color=team_colors[row['팀명']]),# 승률에 따라 색상을 다르게 설정
    ))

# 막대 간격을 넓히기 위해 레이아웃을 조정
fig.update_layout(
    barmode='relative',
    bargap=0.5,  # 막대 간격 설정
    title_text='팀별 패배 비교',  # 그래프 제목
    xaxis=dict(title='팀명'),  # x축 제목
    yaxis=dict(title='패배'),  # y축 제목
    showlegend=True,
    width = 400
)

# 그래프 보여주기
fig.show()

In [165]:
fig = go.Figure()

temp = team_pitch.copy()
temp = temp.loc[temp['이닝'] > 30]
temp['팀명'] = temp['팀명'].apply(lambda x: '한화' if x == '한화' else '타 팀')
temp = temp.groupby(['팀명', '포지션']).mean().reset_index()
temp = temp.loc[temp['포지션'] == '중계']
# 막대 추가
for index, row in temp.iterrows():
    fig.add_trace(go.Bar(
        x=[row['팀명']],
        y=[row['실점']],
        name=row['팀명'],
        text=row['실점']/row['이닝'],
        textposition='auto',
        texttemplate='%{text:.2f}',
        marker=dict(color=team_colors[row['팀명']]),# 승률에 따라 색상을 다르게 설정
    ))

# 막대 간격을 넓히기 위해 레이아웃을 조정
fig.update_layout(
    barmode='relative',
    bargap=0.5,  # 막대 간격 설정
    title_text='팀별 실점 비교',  # 그래프 제목
    xaxis=dict(title='팀명'),  # x축 제목
    yaxis=dict(title='이닝 대비 실점'),  # y축 제목
    showlegend=True,
    width = 400
)

# 그래프 보여주기
fig.show()

In [142]:
fig = go.Figure()

temp = team_pitch.copy()
temp['팀명'] = temp['팀명'].apply(lambda x: '한화' if x == '한화' else '타 팀')
temp = temp.groupby(['팀명', '포지션']).mean().reset_index()
temp = temp.loc[temp['포지션'] == '중계']
# 막대 추가
for index, row in temp.iterrows():
    fig.add_trace(go.Bar(
        x=[row['팀명']],
        y=[row['FIP']],
        name=row['팀명'],
        text=row['FIP'],
        textposition='auto',
        texttemplate='%{text:.2f}',
        marker=dict(color=team_colors[row['팀명']]),# 승률에 따라 색상을 다르게 설정
    ))

# 막대 간격을 넓히기 위해 레이아웃을 조정
fig.update_layout(
    barmode='relative',
    bargap=0.5,  # 막대 간격 설정
    title_text='팀별 FIP 비교',  # 그래프 제목
    xaxis=dict(title='팀명'),  # x축 제목
    yaxis=dict(title='FIP'),  # y축 제목
    showlegend=True,
    width = 400
)

# 그래프 보여주기
fig.show()

In [180]:
team_pitch[['팀명', '포지션', '홀드']]

,팀명,포지션,홀드
0,KIA,선발,0
1,KIA,중계,66
2,KT,선발,6
3,KT,중계,54
4,LG,선발,4
5,LG,중계,85
6,NC,선발,0
7,NC,중계,88
8,SSG,선발,12
9,SSG,중계,59


In [182]:
fig = go.Figure()

temp = team_pitch.sort_values(by='홀드', ascending=False)
temp['팀명'] = temp['팀명'].apply(lambda x: '한화' if x == '한화' else '타 팀')
temp = temp.loc[temp['포지션'] == '중계']
temp = temp.groupby('팀명').agg({'홀드' : 'mean'}).reset_index()
print(temp)

# 막대 추가
for index, row in temp.iterrows():
    fig.add_trace(go.Bar(
        x=[row['팀명']],
        y=[row['홀드']],
        name=row['팀명'],
        text=row['홀드'],
        textposition='auto',
        texttemplate='%{text:.2f}',
        marker=dict(color=team_colors[row['팀명']]),# 승률에 따라 색상을 다르게 설정
    ))

# 막대 간격을 넓히기 위해 레이아웃을 조정
fig.update_layout(
    barmode='relative',
    bargap=0.5,  # 막대 간격 설정
    title_text='팀별 홀드 비교',  # 그래프 제목
    xaxis=dict(title='팀명'),  # x축 제목
    yaxis=dict(title='홀드'),  # y축 제목
    showlegend=True,
    width = 500
)

# 그래프 보여주기
fig.show()

    팀명         홀드
0  타 팀  66.555556
1   한화  61.000000


In [183]:
fig = go.Figure()

temp = team_pitch.sort_values(by='홀드', ascending=False)
temp['팀명'] = temp['팀명'].apply(lambda x: '한화' if x == '한화' else '타 팀')
temp = temp.loc[temp['포지션'] == '중계']
temp = temp.groupby('팀명').agg({'세이브' : 'mean'}).reset_index()
print(temp)

# 막대 추가
for index, row in temp.iterrows():
    fig.add_trace(go.Bar(
        x=[row['팀명']],
        y=[row['세이브']],
        name=row['팀명'],
        text=row['세이브'],
        textposition='auto',
        texttemplate='%{text:.2f}',
        marker=dict(color=team_colors[row['팀명']]),# 승률에 따라 색상을 다르게 설정
    ))

# 막대 간격을 넓히기 위해 레이아웃을 조정
fig.update_layout(
    barmode='relative',
    bargap=0.5,  # 막대 간격 설정
    title_text='팀별 세이브 비교',  # 그래프 제목
    xaxis=dict(title='팀명'),  # x축 제목
    yaxis=dict(title='세이브'),  # y축 제목
    showlegend=True,
    width = 500
)

# 그래프 보여주기
fig.show()

    팀명        세이브
0  타 팀  32.777778
1   한화  19.000000


In [43]:
hitter2023 = pd.read_csv('hitter2023.csv', index_col=False)
hitter2023 = hitter2023.groupby('팀').count().reset_index()
hitter2023 = hitter2023[['팀', 'wRC+']]
hitter2023 = hitter2023.loc[hitter2023['팀'] != 'LG,키움']
hitter2023

,팀,wRC+
0,KIA,9
1,KT,6
2,LG,7
4,NC,6
5,SSG,7
6,두산,6
7,롯데,9
8,삼성,5
9,키움,7
10,한화,5


In [44]:
hitter2023

,팀,wRC+
0,KIA,9
1,KT,6
2,LG,7
4,NC,6
5,SSG,7
6,두산,6
7,롯데,9
8,삼성,5
9,키움,7
10,한화,5


In [45]:
fig = go.Figure()


hitter2023 = hitter2023.sort_values('wRC+', ascending=False)

# 막대 추가
for index, row in hitter2023.iterrows():
    fig.add_trace(go.Bar(
        x=[row['팀']],
        y=[row['wRC+']],
        name=row['팀'],
        marker=dict(color=team_colors[row['팀']]),# 승률에 따라 색상을 다르게 설정
    ))

# 막대 간격을 넓히기 위해 레이아웃을 조정
fig.update_layout(
    barmode='relative',
    bargap=0.5,  # 막대 간격 설정
    title_text='팀별 WRC+ 100이상 선수 명 수',  # 그래프 제목
    xaxis=dict(title='팀'),  # x축 제목
    yaxis=dict(title='wRC+'),  # y축 제목
    showlegend=True,
    width = 1000
)

# 그래프 보여주기
fig.show()

In [46]:
import kaleido
print(kaleido.__version__)

0.2.1


In [47]:
def plot_hanwha_horizontal_comparison(df, metric_column):
    """
    주어진 DataFrame에서 지정된 지표를 기준으로 한화 팀이 각 포지션별로 다른 팀들의 평균에 비해 어느 정도 뒤처지거나 앞서는지를 수평 막대 그래프로 나타냅니다.
    양수는 파스텔 청색 계열, 음수는 파스텔 적색 계열로 표시합니다.

    :param df: DataFrame
    :param metric_column: 비교할 지표의 컬럼명
    """
    positions = df['포지션'].unique()
    differences = []

    for position in positions:
        # 각 포지션별로 모든 팀의 평균 계산
        position_avg = df[(df['포지션'] == position) & df['팀명'] != '한화'][metric_column].mean()

        # 한화 팀의 해당 지표 값
        hanwha_metric = df[(df['팀명'] == '한화') & (df['포지션'] == position)][metric_column].values[0]

        # 한화 팀과 평균의 차이를 비율로 계산
        print(position_avg, hanwha_metric)
        difference_ratio = (hanwha_metric - position_avg) / position_avg
        differences.append((position, difference_ratio))

    # 차이를 DataFrame으로 변환
    diff_df = pd.DataFrame(differences, columns=['포지션', '차이 비율'])

    # 색상 지정
    diff_df['색상'] = ['Negative' if x < 0 else 'Positive' for x in diff_df['차이 비율']]

    # Plotly 막대 그래프 생성
    fig = px.bar(
        diff_df, 
        x='차이 비율', 
        y='포지션', 
        text='차이 비율', 
        color='색상',
        color_discrete_map={'Negative': 'lightcoral', 'Positive': 'skyblue'},
        orientation='h',
        title=f'포지션별 타 팀 평균 대비 한화 지표 평가 ({metric_column})',
        width=1000,
        height= 300
    )

    # 각 막대에 수치 추가 및 레이아웃 설정
    
    fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
    fig.update_layout(uniformtext_minsize=8, showlegend=False, uniformtext_mode='hide', xaxis_tickformat='.2f')
    
    path_name = './plot/' + str(metric_column)+ '비교.png'
    fig.write_image(path_name)
    fig.show()

In [48]:
plot_hanwha_horizontal_comparison(team_hit, '타율')
plot_hanwha_horizontal_comparison(team_hit, '장타율')
plot_hanwha_horizontal_comparison(team_hit, '출루율')
plot_hanwha_horizontal_comparison(team_WAR, 'WAR*')
plot_hanwha_horizontal_comparison(team_hit, '효율')

0.2619325842696629 0.248
0.2619325842696629 0.222
0.2619325842696629 0.298
0.2619325842696629 0.244
0.2619325842696629 0.226
0.2619325842696629 0.226
0.2619325842696629 0.243
0.2619325842696629 0.153
0.2619325842696629 0.231


0.37206741573033714 0.394
0.37206741573033714 0.268
0.37206741573033714 0.541
0.37206741573033714 0.362
0.37206741573033714 0.289
0.37206741573033714 0.347
0.37206741573033714 0.315
0.37206741573033714 0.167
0.37206741573033714 0.283


0.33724719101123596 0.332
0.33724719101123596 0.333
0.33724719101123596 0.388
0.33724719101123596 0.329
0.33724719101123596 0.294
0.33724719101123596 0.282
0.33724719101123596 0.314
0.33724719101123596 0.208
0.33724719101123596 0.35


2.382247191011236 2.31
2.382247191011236 0.84
2.382247191011236 6.93
2.382247191011236 2.09
2.382247191011236 0.72
2.382247191011236 0.13000000000000006
2.382247191011236 1.34
2.382247191011236 -0.79
2.382247191011236 2.4299999999999997


0.1251123595505618 0.149
0.1251123595505618 0.077
0.1251123595505618 0.196
0.1251123595505618 0.122
0.1251123595505618 0.054
0.1251123595505618 0.124
0.1251123595505618 0.097
0.1251123595505618 0.069
0.1251123595505618 0.096


In [49]:
hitter.columns

Index(['이름', 'id', '연도', '팀명', '타율', '경기', '타석', '타수', '득점', '안타', '2루타',
       '3루타', '홈런', '루타', '타점', '도루', '도루실패', '볼넷', '사구', '삼진', '병살타', '장타율',
       '출루율', '실책', '생년월일', '포지션', '희생플라이', 'WAR*'],
      dtype='object')

In [50]:
hitter.loc[(hitter['팀명'] == '한화') & (hitter['포지션'] == '포수')]

,이름,id,연도,팀명,타율,경기,타석,타수,득점,안타,...,사구,삼진,병살타,장타율,출루율,실책,생년월일,포지션,희생플라이,WAR*
174,박상언,66707,2023,한화,0.200,86,161,145,13,29,...,1,38,6,0.269,0.253,1,1997-03-03,포수,2.0,-0.23
199,이재용,67993,2023,한화,0.500,2,2,2,1,1,...,0,1,0,0.500,0.500,0,1999-02-28,포수,0.0,0.02
240,허관회,69706,2023,한화,0.000,10,7,7,1,0,...,0,2,0,0.000,0.000,1,1999-02-12,포수,0.0,-0.16
273,최재훈,78288,2023,한화,0.248,125,417,327,23,81,...,23,48,9,0.294,0.392,3,1989-08-27,포수,2.0,2.80


In [51]:
player_name = '채은성'

player_data = hitter[hitter['이름'] == player_name]

# 각 지표별 순위를 계산합니다.
ranking_hitter = hitter.rank(ascending=False)  # 높은 값이 1등이 되도록 내림차순으로 순위를 매김

# 선수의 각 지표별 순위를 저장할 딕셔너리를 생성합니다.
player_rankings = {}

# 순위를 계산할 지표들을 선택합니다.
metrics = ['타율', '경기', '타석', '타수', '득점', '타점', '안타', '2루타', '3루타', '루타', '홈런', '도루', '볼넷', '실책', '삼진', '병살타', '장타율', '출루율', 'WAR*']

# 각 지표별로 순위를 계산하여 딕셔너리에 저장합니다.
for metric in metrics:
    player_rank = ranking_hitter.loc[hitter['이름'] == player_name, metric].iloc[0]
    player_rankings[metric] = player_rank

# 결과 출력
print(f"{player_name} 선수의 각 지표별 순위:")
for metric, rank in player_rankings.items():
    print(f"{metric}: {rank}위")

채은성 선수의 각 지표별 순위:
타율: 89.5위
경기: 11.5위
타석: 6.0위
타수: 7.0위
득점: 19.0위
타점: 10.0위
안타: 20.0위
2루타: 50.5위
3루타: 196.5위
루타: 12.0위
홈런: 3.5위
도루: 225.0위
볼넷: 25.5위
실책: 63.0위
삼진: 11.0위
병살타: 5.5위
장타율: 39.0위
출루율: 68.5위
WAR*: 34.0위


In [52]:
player_name = '최재훈'

position_hitter = hitter.loc[hitter['포지션'] == '포수']

player_data = position_hitter.loc[hitter['이름'] == player_name]

# 각 지표별 순위를 계산합니다.
ranking_hitter = position_hitter.rank(ascending=False)  # 높은 값이 1등이 되도록 내림차순으로 순위를 매김

# 선수의 각 지표별 순위를 저장할 딕셔너리를 생성합니다.
player_rankings = {}

# 순위를 계산할 지표들을 선택합니다.
metrics = ['타율', '경기', '타석', '타수', '득점', '타점', '안타', '2루타', '3루타', '루타', '홈런', '도루', '볼넷', '삼진', '실책', '병살타', '장타율', '출루율', 'WAR*']

# 각 지표별로 순위를 계산하여 딕셔너리에 저장합니다.
for metric in metrics:
    player_rank = ranking_hitter.loc[hitter['이름'] == player_name, metric].iloc[0]
    player_rankings[metric] = player_rank

# 결과 출력
print(f"{player_name} 선수의 각 지표별 순위:")
for metric, rank in player_rankings.items():
    print(f"{metric}: {rank}위")

최재훈 선수의 각 지표별 순위:
타율: 14.0위
경기: 4.5위
타석: 5.0위
타수: 6.0위
득점: 9.5위
타점: 8.0위
안타: 6.0위
2루타: 6.0위
3루타: 26.0위
루타: 6.0위
홈런: 20.5위
도루: 9.0위
볼넷: 2.0위
삼진: 8.0위
실책: 12.5위
병살타: 6.0위
장타율: 20.0위
출루율: 4.0위
WAR*: 5.0위


In [53]:
def plot_team_comparison(df, metric_column):
    """
    주어진 DataFrame에서 지정된 지표를 기준으로 한화 팀이 전체 팀 평균에 비해 어느 정도 뒤처지거나 앞서는지를 수평 막대 그래프로 나타냅니다.
    색상은 팀별로 정해진 색상 코드를 사용합니다.

    :param df: DataFrame
    :param metric_column: 비교할 지표의 컬럼명
    """
    # 전체 팀의 평균 계산
    overall_avg = df.loc[df['팀명'] != '한화'][metric_column].mean()

    # 한화 팀의 해당 지표 값
    hanwha_metric = df.loc[df['팀명'] == '한화'][metric_column].values[0]

    # 한화 팀과 전체 평균의 차이를 비율로 계산
    #print(hanwha_metric, overall_avg)
    print(overall_avg, hanwha_metric)
    difference_ratio = (hanwha_metric - overall_avg) / overall_avg

    # 차이 비율을 담은 DataFrame 생성
    diff_df = pd.DataFrame({'팀명': ['한화'], '차이 비율': [difference_ratio]})
    diff_df['색상'] = ['Negative' if x < 0 else 'Positive' for x in diff_df['차이 비율']]

    # Plotly 막대 그래프 생성
    fig = px.bar(
        diff_df,
        x='차이 비율',
        y='팀명',
        text='차이 비율',
        color='색상',
        orientation='h',
        title=f'전체 팀 평균 대비 한화 {metric_column} 지표 평가',
        width=1200,
        height=300
        
    )
    

    # 각 막대에 수치 추가 및 레이아웃 설정
    fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
    fig.update_layout(uniformtext_minsize=8, showlegend=False, uniformtext_mode='hide', xaxis_tickformat='.2f', xaxis_range = [-1, 1], yaxis_range = [-2, 2])
    fig.show()

In [54]:
plot_team_comparison(total_hit, '출루율')
temp = hitter.loc[(hitter['포지션'] != '1루수') & (hitter['포지션'] != '3루수'), ['팀명', '타석', '타수', '득점', '안타', '2루타', '3루타', '도루', '홈런', '루타', '타점', '볼넷', '사구', '삼진', '병살타', '희생플라이']]
temp = temp.groupby('팀명').sum().reset_index()
temp['타율'] = (temp['안타'] / temp['타수']).round(3)
temp['1루타'] = temp['안타'] - temp['2루타'] - temp['3루타'] - temp['홈런']
temp['장타율'] = ((total_hit['1루타'] + 2*temp['2루타'] + 3*temp['3루타'] + 4*temp['홈런']) / temp['타수']).round(3)
temp['출루율'] = ((temp['안타'] + temp['볼넷'] + temp['사구']) / (temp['타수'] + temp['볼넷'] + temp['사구'] + temp['희생플라이'])).round(3)
temp['효율'] = (temp['타점'] / temp['타수']).round(3)
plot_team_comparison(temp, '출루율')

0.34011111111111114 0.324


0.34344444444444444 0.313


In [55]:
plot_team_comparison(total_hit, '장타율')
temp = hitter.loc[(hitter['포지션'] != '1루수') & (hitter['포지션'] != '3루수'), ['팀명', '타석', '타수', '득점', '안타', '2루타', '3루타', '도루', '홈런', '루타', '타점', '볼넷', '사구', '삼진', '병살타', '희생플라이']]
temp = temp.groupby('팀명').sum().reset_index()
temp['타율'] = (temp['안타'] / temp['타수']).round(3)
temp['1루타'] = temp['안타'] - temp['2루타'] - temp['3루타'] - temp['홈런']
temp['장타율'] = ((total_hit['1루타'] + 2*temp['2루타'] + 3*temp['3루타'] + 4*temp['홈런']) / temp['타수']).round(3)
temp['출루율'] = ((temp['안타'] + temp['볼넷'] + temp['사구']) / (temp['타수'] + temp['볼넷'] + temp['사구'] + temp['희생플라이'])).round(3)
temp['효율'] = (temp['타점'] / temp['타수']).round(3)
plot_team_comparison(temp, '장타율')

0.3765555555555556 0.35


0.4334444444444444 0.38


In [56]:
plot_team_comparison(total_hit, '타율')
temp = hitter.loc[(hitter['포지션'] != '1루수') & (hitter['포지션'] != '3루수'), ['팀명', '타석', '타수', '득점', '안타', '2루타', '3루타', '도루', '홈런', '루타', '타점', '볼넷', '사구', '삼진', '병살타', '희생플라이']]
temp = temp.groupby('팀명').sum().reset_index()
temp['타율'] = (temp['안타'] / temp['타수']).round(3)
temp['1루타'] = temp['안타'] - temp['2루타'] - temp['3루타'] - temp['홈런']
temp['장타율'] = ((total_hit['1루타'] + 2*temp['2루타'] + 3*temp['3루타'] + 4*temp['홈런']) / temp['타수']).round(3)
temp['출루율'] = ((temp['안타'] + temp['볼넷'] + temp['사구']) / (temp['타수'] + temp['볼넷'] + temp['사구'] + temp['희생플라이'])).round(3)
temp['효율'] = (temp['타점'] / temp['타수']).round(3)
plot_team_comparison(temp, '타율')

0.266 0.241


0.26888888888888896 0.232


In [57]:
plot_team_comparison(team_hit, '득점')
temp = team_hit.loc[(team_hit['포지션'] != '1루수') & (team_hit['포지션'] != '3루수')]
plot_team_comparison(temp, '득점')

75.2125 105


75.46774193548387 50


In [58]:
def plot_hanwha_horizontal_comparison2(df, metric_column):
    """
    주어진 DataFrame에서 지정된 지표를 기준으로 한화 팀이 각 포지션별로 다른 팀들의 평균에 비해 어느 정도 뒤처지거나 앞서는지를 수평 막대 그래프로 나타냅니다.
    양수는 파스텔 청색 계열, 음수는 파스텔 적색 계열로 표시합니다.

    :param df: DataFrame
    :param metric_column: 비교할 지표의 컬럼명
    """
    positions = df['포지션'].unique()
    differences = []

    for position in positions:
        # 각 포지션별로 모든 팀의 평균 계산
        position_avg = df[(df['포지션'] == position) & df['팀명'] != '한화'][metric_column].mean()

        # 한화 팀의 해당 지표 값
        hanwha_metric = df[(df['팀명'] == '한화') & (df['포지션'] == position)][metric_column].values[0]

        # 한화 팀과 평균의 차이를 비율로 계산
        print(position, hanwha_metric, position_avg)
        difference_ratio = (hanwha_metric - position_avg) / position_avg
        differences.append((position, difference_ratio))

    # 차이를 DataFrame으로 변환
    diff_df = pd.DataFrame(differences, columns=['포지션', '차이 비율'])

    # 색상 지정
    diff_df['색상'] = ['Positive' if x < 0 else 'Negative' for x in diff_df['차이 비율']]

    # Plotly 막대 그래프 생성
    fig = px.bar(
        diff_df, 
        x='차이 비율', 
        y='포지션', 
        text='차이 비율', 
        color='색상',
        color_discrete_map={'Negative': 'lightcoral', 'Positive': 'skyblue'},
        orientation='h',
        title=f'포지션별 타 팀 평균 대비 한화 지표 평가 ({metric_column})',
        width=1000,
        height= 300
    )

    # 각 막대에 수치 추가 및 레이아웃 설정
    
    fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
    fig.update_layout(uniformtext_minsize=8, showlegend=False, uniformtext_mode='hide', xaxis_tickformat='.2f')
    
    path_name = './plot/' + str(metric_column)+ '비교.png'
    fig.write_image(path_name)
    fig.show()

In [59]:
team_pitch.columns

Index(['팀명', '포지션', '경기', '완투', '완봉', '승리', '패배', '세이브', '홀드', '타자수', '이닝',
       '피안타', '피홈런', '볼넷', '사구', '삼진', '실점', '자책점', 'WPA', '승률', '평균자책점',
       'FIP', 'WHIP'],
      dtype='object')

In [60]:
team_pitch

,팀명,포지션,경기,완투,완봉,승리,패배,세이브,홀드,타자수,...,볼넷,사구,삼진,실점,자책점,WPA,승률,평균자책점,FIP,WHIP
0,KIA,선발,155,1,0,44,49,0,0,3315,...,307,29,616,409,369,-3.15,0.473,4.39,4.054134,1.419876
1,KIA,중계,519,0,0,29,20,33,66,2051,...,223,27,334,207,181,0.31,0.592,3.42,4.333678,1.377283
2,KT,선발,228,0,0,59,43,0,6,3810,...,258,30,654,415,380,2.56,0.578,3.85,3.689460,1.338583
3,KT,중계,309,0,0,20,19,37,54,1392,...,115,14,259,147,133,4.54,0.513,3.65,3.842417,1.289241
4,LG,선발,204,0,0,61,36,3,4,3815,...,278,40,611,428,374,0.25,0.629,3.81,3.831591,1.339518
5,LG,중계,446,0,0,31,24,32,85,1790,...,192,36,383,168,150,3.03,0.564,3.22,3.887159,1.272641
6,NC,선발,177,0,0,48,39,0,0,3135,...,257,34,611,341,301,2.23,0.552,3.68,4.043273,1.256957
7,NC,중계,488,0,0,23,26,33,88,2032,...,220,24,412,228,203,0.60,0.469,3.88,4.003100,1.368175
8,SSG,선발,216,1,0,46,49,1,12,3737,...,364,47,608,471,425,-2.75,0.484,4.49,4.519601,1.480742
9,SSG,중계,389,0,0,30,16,45,59,1798,...,219,16,341,192,177,3.80,0.652,3.95,4.233309,1.552965


In [61]:
team_defense2 = team_defense.loc[team_defense['포지션'] != '지명타자']
plot_hanwha_horizontal_comparison2(team_defense2, '실책')

1루수 10 11.9375
2루수 8 11.9375
3루수 19 11.9375
우익수 8 11.9375
유격수 23 11.9375
좌익수 8 11.9375
중견수 12 11.9375
포수 5 11.9375


In [62]:
plot_hanwha_horizontal_comparison2(team_pitch, '평균자책점')
plot_hanwha_horizontal_comparison2(team_pitch, 'FIP')
#plot_hanwha_horizontal_comparison(team_pitch, '승률')
#plot_hanwha_horizontal_comparison(team_WAR_p, 'WAR')

선발 4.25 4.0565
중계 3.71 4.0565


선발 4.136587677725118 4.087776564035363
중계 4.069556251631428 4.087776564035363


,팀명,포지션,경기,완투,완봉,승리,패배,세이브,홀드,타자수,...,볼넷,사구,삼진,실점,자책점,WPA,승률,평균자책점,FIP,WHIP
0,KIA,선발,155,1,0,44,49,0,0,3315,...,307,29,616,409,369,-3.15,0.473,4.39,4.054134,1.419876
1,KIA,중계,519,0,0,29,20,33,66,2051,...,223,27,334,207,181,0.31,0.592,3.42,4.333678,1.377283
2,KT,선발,228,0,0,59,43,0,6,3810,...,258,30,654,415,380,2.56,0.578,3.85,3.689460,1.338583
3,KT,중계,309,0,0,20,19,37,54,1392,...,115,14,259,147,133,4.54,0.513,3.65,3.842417,1.289241
4,LG,선발,204,0,0,61,36,3,4,3815,...,278,40,611,428,374,0.25,0.629,3.81,3.831591,1.339518
5,LG,중계,446,0,0,31,24,32,85,1790,...,192,36,383,168,150,3.03,0.564,3.22,3.887159,1.272641
6,NC,선발,177,0,0,48,39,0,0,3135,...,257,34,611,341,301,2.23,0.552,3.68,4.043273,1.256957
7,NC,중계,488,0,0,23,26,33,88,2032,...,220,24,412,228,203,0.60,0.469,3.88,4.003100,1.368175
8,SSG,선발,216,1,0,46,49,1,12,3737,...,364,47,608,471,425,-2.75,0.484,4.49,4.519601,1.480742
9,SSG,중계,389,0,0,30,16,45,59,1798,...,219,16,341,192,177,3.80,0.652,3.95,4.233309,1.552965


In [64]:
pitcher.groupby('팀명').agg({'홀드' : 'sum', '세이브' : 'sum'}).reset_index()

# 72, 36.8

,팀명,홀드,세이브
0,KIA,66,33
1,KT,60,37
2,LG,89,35
3,NC,88,33
4,SSG,71,46
5,두산,65,41
6,롯데,81,35
7,삼성,72,38
8,키움,56,33
9,한화,68,19


In [65]:
team_pitch

,팀명,포지션,경기,완투,완봉,승리,패배,세이브,홀드,타자수,...,볼넷,사구,삼진,실점,자책점,WPA,승률,평균자책점,FIP,WHIP
0,KIA,선발,155,1,0,44,49,0,0,3315,...,307,29,616,409,369,-3.15,0.473,4.39,4.054134,1.419876
1,KIA,중계,519,0,0,29,20,33,66,2051,...,223,27,334,207,181,0.31,0.592,3.42,4.333678,1.377283
2,KT,선발,228,0,0,59,43,0,6,3810,...,258,30,654,415,380,2.56,0.578,3.85,3.689460,1.338583
3,KT,중계,309,0,0,20,19,37,54,1392,...,115,14,259,147,133,4.54,0.513,3.65,3.842417,1.289241
4,LG,선발,204,0,0,61,36,3,4,3815,...,278,40,611,428,374,0.25,0.629,3.81,3.831591,1.339518
5,LG,중계,446,0,0,31,24,32,85,1790,...,192,36,383,168,150,3.03,0.564,3.22,3.887159,1.272641
6,NC,선발,177,0,0,48,39,0,0,3135,...,257,34,611,341,301,2.23,0.552,3.68,4.043273,1.256957
7,NC,중계,488,0,0,23,26,33,88,2032,...,220,24,412,228,203,0.60,0.469,3.88,4.003100,1.368175
8,SSG,선발,216,1,0,46,49,1,12,3737,...,364,47,608,471,425,-2.75,0.484,4.49,4.519601,1.480742
9,SSG,중계,389,0,0,30,16,45,59,1798,...,219,16,341,192,177,3.80,0.652,3.95,4.233309,1.552965


In [66]:
plot_hanwha_horizontal_comparison(team_pitch, '승리')
plot_hanwha_horizontal_comparison(team_pitch, '패배')
plot_hanwha_horizontal_comparison(team_WAR_p, 'WAR')
plot_hanwha_horizontal_comparison(team_pitch, '홀드')
plot_hanwha_horizontal_comparison(team_pitch, '세이브')

35.4 36
35.4 22


35.4 61
35.4 19


8.5485 9.1
8.5485 6.5


35.8 7
35.8 61


17.5 0
17.5 19


In [67]:
pitcher.loc[(pitcher['팀명'] == '한화') & (pitcher['포지션'] == '선발')]

,이름,id,연도,팀명,평균자책점,경기,완투,완봉,승리,패배,...,생년월일,포지션,WAR,폭투,ERA,FIP,WHIP,ERA+,FIP+,WPA
12,남지민,50720,2023,한화,6.45,16,0,0,1,7,...,2001-02-12,선발,-0.52,3.0,6.45,3.66,1.78,64.4,113.6,-1.29
13,한승주,50726,2023,한화,3.95,47,0,0,1,4,...,2001-03-17,선발,1.02,5.0,3.95,4.39,1.59,105.2,94.5,-0.76
25,김기중,51715,2023,한화,4.63,37,0,0,1,3,...,2002-11-16,선발,0.28,2.0,4.63,4.70,1.54,89.6,88.3,0.20
43,문동주,52701,2023,한화,3.72,23,0,0,8,8,...,2003-12-23,선발,2.51,5.0,3.72,3.66,1.31,111.7,113.2,-0.08
44,페냐,52720,2023,한화,3.60,32,0,0,11,11,...,1990-01-25,선발,3.53,16.0,3.60,4.10,1.17,115.2,101.3,-0.27
58,산체스,53714,2023,한화,3.79,24,0,0,7,8,...,1997-04-11,선발,1.79,3.0,3.79,3.97,1.30,109.7,104.4,0.00
72,이태양,60768,2023,한화,3.23,50,0,0,3,3,...,1990-07-03,선발,2.01,1.0,3.23,3.57,1.20,128.6,116.1,0.65
80,한승혁,61666,2023,한화,6.44,21,0,0,0,3,...,1993-01-03,선발,-0.34,4.0,6.44,4.96,1.90,64.5,83.8,-1.69
121,김민우,65764,2023,한화,6.97,12,0,0,1,6,...,1995-07-25,선발,-1.06,5.0,6.97,5.03,1.80,59.6,82.5,-1.61
200,장민재,79764,2023,한화,4.83,25,0,0,3,8,...,1990-03-19,선발,-0.12,5.0,4.83,5.02,1.46,86.0,82.7,-0.54


In [68]:
pitcher.loc[(pitcher['팀명'] == '한화') & (pitcher['포지션'] == '중계')]

,이름,id,연도,팀명,평균자책점,경기,완투,완봉,승리,패배,...,생년월일,포지션,WAR,폭투,ERA,FIP,WHIP,ERA+,FIP+,WPA
11,강재민,50705,2023,한화,6.44,43,0,0,1,3,...,1997-04-03,중계,-0.37,0.0,6.44,4.87,1.25,64.5,85.2,-1.03
24,김규연,51713,2023,한화,2.75,23,0,0,0,1,...,2002-08-23,중계,0.54,2.0,2.75,3.24,1.32,151.2,127.9,0.02
92,윤대경,63464,2023,한화,2.45,47,0,0,5,1,...,1994-04-09,중계,1.11,2.0,2.45,4.93,1.15,169.2,84.2,0.58
116,이민우,65616,2023,한화,2.63,17,0,0,2,1,...,1993-02-09,중계,0.26,1.0,2.63,5.05,1.02,157.6,82.1,0.07
120,주현상,65707,2023,한화,1.96,55,0,0,2,2,...,1992-08-10,중계,2.58,0.0,1.96,3.22,0.84,211.7,128.5,1.41
122,김범수,65769,2023,한화,4.19,76,0,0,5,5,...,1995-10-03,중계,0.77,4.0,4.19,3.89,1.56,99.2,106.5,-0.32
140,박상원,67703,2023,한화,3.65,55,0,0,5,3,...,1994-09-09,중계,0.37,2.0,3.65,3.38,1.49,113.8,122.6,0.92
184,정우람,74857,2023,한화,5.36,52,0,0,0,1,...,1985-06-01,중계,0.24,3.0,5.36,4.34,1.54,77.5,95.7,0.24
192,장시환,77318,2023,한화,3.38,39,0,0,2,2,...,1987-11-01,중계,1.00,3.0,3.38,4.51,1.53,123.0,92.0,-0.81


In [69]:
for_regression = pitcher[(pitcher['이름'] != '문경찬')]
for_regression = for_regression[(for_regression['이름'] != '김영준')]
for_regression = for_regression[(for_regression['이름'] != '최하늘')]
for_regression = for_regression[(for_regression['이름'] != '이영준')]
for_regression = for_regression[(for_regression['이름'] != '김동규')]


for_regression = for_regression.loc[for_regression['이닝'] > 30]
for_regression['색상'] = for_regression['팀명'].apply(lambda x: 'orange' if x == '한화' else 'blue')
for_regression

,이름,id,연도,팀명,평균자책점,경기,완투,완봉,승리,패배,...,포지션,WAR,폭투,ERA,FIP,WHIP,ERA+,FIP+,WPA,색상
0,유영찬,50106,2023,LG,3.44,67,0,0,6,3,...,중계,1.55,8.0,3.44,4.58,1.40,118.6,89.5,-0.12,blue
2,김윤식,50157,2023,LG,4.22,17,0,0,6,4,...,선발,1.00,4.0,4.22,4.14,1.49,96.7,98.9,0.40,blue
4,김동혁,50360,2023,키움,7.32,35,0,0,1,7,...,선발,-0.81,0.0,7.55,4.38,1.75,54.4,93.7,-1.16,blue
5,뷰캐넌,50404,2023,삼성,2.54,30,1,0,12,8,...,선발,5.10,7.0,2.54,3.01,1.15,166.9,138.1,2.12,blue
6,황동재,50441,2023,삼성,7.11,7,0,0,0,5,...,선발,-0.58,0.0,7.11,6.38,1.93,59.6,66.0,-0.83,blue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,양현종,77637,2023,KIA,3.58,29,0,0,9,11,...,선발,2.80,5.0,3.53,3.72,1.33,117.2,110.4,-0.03,blue
195,김광현,77829,2023,SSG,3.53,30,0,0,9,8,...,선발,3.30,5.0,3.53,4.16,1.38,118.6,101.3,0.92,blue
197,정찬헌,78148,2023,키움,4.75,14,1,0,2,8,...,선발,0.67,0.0,4.75,3.78,1.17,86.5,108.1,-0.15,blue
198,임창민,78352,2023,키움,2.51,51,0,0,2,2,...,중계,1.89,0.0,2.51,3.59,1.46,163.8,113.1,0.55,blue


In [70]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go


# 한화 선수들에 대한 색상 지정
for_regression['팀 색상'] = for_regression['팀명'].apply(lambda x: 'orange' if x == '한화' else 'blue')

# 한화 선수들과 한화가 아닌 선수들의 산점도 생성
fig = px.scatter(
    for_regression, 
    x='ERA', 
    y='FIP', 
    color='팀 색상',  # 색상을 결정하는 열의 이름을 전달
    trendline='ols'
)

# 레이아웃 설정
fig.update_layout(
    xaxis_title='ERA',
    yaxis_title='FIP',
    title='한화 선수 대비 다른 팀 선수의 ERA 대비 FIP',
    showlegend=True,
    width=800,
    height=800
)

# 그래프 출력
fig.show()


In [77]:
#non_hanwha_slope, hanwha_slope, non_hanwha_pvalue, hanwha_pvalue

In [78]:
hanwha_players = for_regression[for_regression['팀명'] == '한화']
hanwha_players['color'] = '#FF6600'

# 한화가 아닌 선수들 필터링
non_hanwha_players = for_regression[for_regression['팀명'] != '한화']

# 한화가 아닌 선수들의 산점도와 회귀선 생성
fig_non_hanwha = px.scatter(
    non_hanwha_players, 
    x='FIP', 
    y='ERA', 
    trendline='ols'
)

# 한화 선수들의 산점도 추가 (주황색 점)
fig_hanwha = px.scatter(
    hanwha_players,
    x='FIP',
    y='ERA',
    color='color',
    trendline='ols',
    trendline_color_override='#FF6600'
)

# 회귀선 결과 추출
non_hanwha_regression_results = px.get_trendline_results(fig_non_hanwha)
hanwha_regression_results = px.get_trendline_results(fig_hanwha)

# 기울기와 y절편 추출
non_hanwha_slope = non_hanwha_regression_results.iloc[0]["px_fit_results"].params[1]
non_hanwha_intercept = non_hanwha_regression_results.iloc[0]["px_fit_results"].params[0]

hanwha_slope = hanwha_regression_results.iloc[0]["px_fit_results"].params[1]
hanwha_intercept = hanwha_regression_results.iloc[0]["px_fit_results"].params[0]

# 두 팀의 회귀선을 한 그래프에 추가
fig = go.Figure()
fig.add_traces(fig_non_hanwha.data + fig_hanwha.data)

# 레이아웃 설정
fig.update_layout(
    xaxis_title='FIP',
    yaxis_title='ERA',
    title='한화 선수 대비 다른 팀 선수의 FIP 대비 ERA',
    showlegend=True,
    width=800,
    height=800
)

# x축, y축 범위 설정
fig.update_xaxes(range=[1, 9])
fig.update_yaxes(range=[1, 9])

# 그래프 출력
fig.show()

# 회귀선의 기울기와 y절편 출력
# 회귀선의 유의성 검정 결과 출력
non_hanwha_pvalue = non_hanwha_regression_results.iloc[0]["px_fit_results"].f_pvalue
hanwha_pvalue = hanwha_regression_results.iloc[0]["px_fit_results"].f_pvalue


/var/folders/g_/dk3yfcbs6kg3ppmw05fqnl2h0000gn/T/ipykernel_3071/1306954520.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [79]:
non_hanwha_slope, hanwha_slope, non_hanwha_pvalue, hanwha_pvalue

(1.1139685651313185,
 1.17302223672556,
 1.4796137610125075e-22,
 0.045950552669740796)

In [80]:
team_pitch.columns

Index(['팀명', '포지션', '경기', '완투', '완봉', '승리', '패배', '세이브', '홀드', '타자수', '이닝',
       '피안타', '피홈런', '볼넷', '사구', '삼진', '실점', '자책점', 'WPA', '승률', '평균자책점',
       'FIP', 'WHIP'],
      dtype='object')

In [81]:
plot_team_comparison(team_pitch, '홀드')
plot_team_comparison(team_pitch, '세이브')

36.0 7


18.38888888888889 0


In [82]:
pitcher.loc[pitcher['팀명'] == '한화', '세이브']

11      0
12      0
13      0
24      1
25      0
43      0
44      0
58      0
72      0
80      0
92      0
116     0
120     0
121     0
122     1
140    16
184     0
192     1
200     0
Name: 세이브, dtype: int64

In [83]:
hitter.loc[(hitter['팀명'] == '') & (hitter['포지션'] == '지명타자')]

,이름,id,연도,팀명,타율,경기,타석,타수,득점,안타,...,사구,삼진,병살타,장타율,출루율,실책,생년월일,포지션,희생플라이,WAR*
